In [65]:
! pip install essentia pandas scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.4 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 51.2 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [72]:
import os
import json
import numpy as np
import pandas as pd
import essentia.standard as es
from tempfile import TemporaryDirectory
from sklearn.preprocessing import StandardScaler

def extract_features_from_json(json_file):
    """
    Extracts relevant features from a JSON file and returns a feature vector.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    # Extract relevant features
    features = [
        data["lowlevel"]["average_loudness"],
        data["lowlevel"]["dissonance"]["mean"],
        data["lowlevel"]["dynamic_complexity"],
        data["lowlevel"]["spectral_centroid"]["mean"],
        data["lowlevel"]["spectral_flux"]["mean"],
        data["lowlevel"]["zerocrossingrate"]["mean"],
        *data["lowlevel"]["barkbands"]["mean"],  # Flatten barkbands
        *data["lowlevel"]["mfcc"]["mean"],      # Flatten MFCCs
        data["rhythm"]["bpm"],
        data["rhythm"]["beats_count"],
        data["rhythm"]["danceability"],
        data["rhythm"]["onset_rate"],
        data["tonal"]["chords_strength"]["mean"],
        data["tonal"]["hpcp_crest"]["mean"],
        data["tonal"]["hpcp_entropy"]["mean"],
        data["tonal"]["key_edma"]["strength"],
        data["tonal"]["key_krumhansl"]["strength"],
        data["tonal"]["key_temperley"]["strength"],
        data["metadata"]["audio_properties"]["length"],
        data["metadata"]["audio_properties"]["sample_rate"]
    ]

    # Convert to a numpy array
    feature_vector = np.array(features)
    return feature_vector

def save_vector_to_csv(vector, artist, song_name, output_folder="song_csv"):
    """
    Saves the feature vector to a CSV file in the specified output folder.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Define the CSV file path
    csv_file = os.path.join(output_folder, f"{artist}-{song_name}.csv")

    # Define column names
    column_names = [
        "average_loudness",
        "dissonance_mean",
        "dynamic_complexity",
        "spectral_centroid_mean",
        "spectral_flux_mean",
        "zerocrossingrate_mean",
        *[f"barkbands_mean_{i}" for i in range(len(vector) - 12)],  # Dynamic column names for barkbands
        *[f"mfcc_mean_{i}" for i in range(13)],  # Dynamic column names for MFCCs
        "bpm",
        "beats_count",
        "danceability",
        "onset_rate",
        "chords_strength_mean",
        "hpcp_crest_mean",
        "hpcp_entropy_mean",
        "key_edma_strength",
        "key_krumhansl_strength",
        "key_temperley_strength",
        "audio_length",
        "sample_rate"]

    # Save the vector to a CSV file
    df = pd.DataFrame([vector])
    df.to_csv(csv_file, index=False, header=False)
    print(f"Feature vector saved to: {csv_file}")

def download_and_extract_song(artist, song_name, audio_folder="audio", csv_folder="song_csv", json_folder="song_features"):
    """
    Downloads a song, extracts features, and saves the feature vector to a CSV file.
    """
    # Ensure audio folder exists
    os.makedirs(audio_folder, exist_ok=True)

    # Format the audio file path
    audio_file = os.path.join(audio_folder, f"{artist}-{song_name}.wav")

    if not os.path.exists(audio_file):
        # If file doesn't exist, download the song using spotdl
        os.system(f"spotdl download '{artist}-{song_name}' --format wav --output '{audio_folder}/'")

        # Rename the downloaded file
        downloaded_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
        if downloaded_files:
            downloaded_file = os.path.join(audio_folder, downloaded_files[0])
            os.rename(downloaded_file, audio_file)

    # Check if file was downloaded
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"Failed to download or locate audio file: {audio_file}")

    # Ensure output folder exists
    os.makedirs(json_folder, exist_ok=True)

    # Define results file per song
    results_file = os.path.join(json_folder, f"{artist}-{song_name}.json")

    # Extract features using Essentia
    features, features_frames = es.MusicExtractor(
        lowlevelStats=['mean', 'stdev'],
        rhythmStats=['mean', 'stdev'],
        tonalStats=['mean', 'stdev'])(audio_file)

    # Save features to JSON
    es.YamlOutput(filename=results_file, format='json')(features)

    # Extract the feature vector from the JSON file
    feature_vector = extract_features_from_json(results_file)

    # Save the feature vector to a CSV file
    save_vector_to_csv(feature_vector, artist, song_name, csv_folder)

    return feature_vector


In [73]:
download_and_extract_song('Lady Gaga', 'Die with a smile')

Feature vector saved to: song_csv/Lady Gaga-Die with a smile.csv


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


array([ 7.30834842e-01,  4.52379614e-01,  5.38787127e+00,  1.24243188e+03,
        7.33930096e-02,  5.87083884e-02,  1.53934699e-04,  5.63180121e-03,
        1.47339527e-03,  9.94074158e-04,  1.71319197e-03,  9.13803757e-04,
        1.22330233e-03,  1.40704459e-03,  2.18217052e-03,  2.93917139e-03,
        1.33293727e-03,  1.79947854e-03,  1.37360161e-03,  8.91951087e-04,
        3.52101517e-04,  2.92371609e-04,  1.96150533e-04,  1.77090697e-04,
        1.41186450e-04,  1.90200153e-04,  1.16518997e-04,  1.21458412e-04,
        1.07300621e-04,  9.79163306e-05,  8.01134520e-05,  2.66523875e-05,
        5.63931735e-06, -6.83442871e+02,  1.28028030e+02, -1.28423948e+01,
       -7.30077386e-01, -6.79468918e+00, -2.13659763e+00,  7.77765393e-01,
        4.36371088e+00, -7.12822616e-01,  3.10417652e+00, -7.53850350e-03,
       -2.64036727e+00, -3.01365876e+00,  1.05055832e+02,  5.39000000e+02,
        1.04016268e+00,  3.17862582e+00,  5.28761566e-01,  1.49700432e+01,
        1.75616884e+00,  